<a href="https://colab.research.google.com/github/juliakreutzer/masakhane-eval/blob/master/TEDTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TED Talk Translation with Masakhane Models

This notebook will load a Masakhane Joey NMT model for your selected language, will translate parts of [TED talks](https://www.ted.com/talks), taken from the multi-target test sets composed [here](http://www.cs.jhu.edu/~kevinduh/a/multitarget-tedtalks/) and then present it to you. 

- Connect to a GPU runtime for fast translations: In the menu, select 'Runtime' -> 'Change Runtime Type' -> 'Hardware Accelarator' -> select 'GPU'.
- Please go through and *execute this notebook cell by cell*. Sometimes cells seem to be empty because the code is hidden. Please execute them anyway. 
- Proceed  until you come to *the post-editing part* where you will be asked to provide your correction to the models' translations. 
- Note that intermediate process won't get saved if the runtime disconnects in the meantime, so make sure you *save the output before you take a break*. You can do that by executing the parts below the post-edit cell *until you've downloaded the files*, and then later indicate the sentence number where you want to resume. 
    - For example, you want to do the first 5 sentences now, the next 5 tomorrow. 
    - Set `start_sent` today to 1, continue with post-editing until you have finished the 5th sentence, and complete the notebook and download the files. 
    - Tomorrow, set `start_sent` to 6 and continue until 10, and so on.
- If you don't know how to translate or correct a sentence, please just give an empty translation as answer.
- If any cells reports an error message that does not let you continue, please take a screenshot and email it to kreutzer@cl.uni-heidelberg.de.
- At the very end, you'll be asked to *download* the created translation files and *send it via email*. 

**Thank you for your time! <3**

## Getting Ready

In [1]:
#@title Installation

! pip3 install gdown --quiet
! pip3 install polyglot --quiet
! pip3 install pyicu --quiet
! pip3 install pycld2 --quiet
! pip3 install morfessor --quiet
! pip3 install pyter3 --quiet

! pip3 install joeynmt


In [2]:
#@title Imports

import os
import re
import yaml
import sacrebleu
import numpy as np
import pandas as pd
import functools
import pyter
import ipywidgets as widgets
from IPython.display import display
from polyglot.text import Text
from subword_nmt import apply_bpe

## Model Preparation

In [3]:
#@title Target language selection

from joeynmt.helpers import load_config

class MasakhaneModelLoader():

  def __init__(self, available_models_file):
    self._model_dir_prefix = 'joeynmt/models/'
    self._src_language = ''
    self.models = self.load_available_models(available_models_file)
  
  def load_available_models(self, available_models_file, 
                            src_language='en', domain='JW300'):
    # Get list of available models.
    # If multiple models: select domain. 
    # Only select relevant models with correct src language.
    models = {}
    with open(available_models_file, 'r') as ofile:
      for i, line in enumerate(ofile):
        entries = line.strip().split("\t")
        if i == 0:
          headers = entries
          header_keys = [h.__str__() for h in headers]
          continue
        model = {h: v for h, v in zip(header_keys, entries)}
        if model['src_language'] != src_language or model['complete'] != 'yes':
          continue
        if model['trg_language'] in models.keys() and model['domain'] != domain:
          continue
        models[model['trg_language']] = model
    print('Found {} Masakhane models.'.format(len(models)))
    self._model_dir_prefix += src_language
    self._src_language = src_language
    return models
  
  def download_model(self, trg_language):
    """ Download model for given trg language. """
    model_dir = "{}-{}".format(self._model_dir_prefix, trg_language)
    !mkdir -p $model_dir
    model_files = self.models[trg_language]
    # Download the checkpoint.
    ckpt_path = os.path.join(model_dir, 'model.ckpt')
    self._download(model_files['ckpt'], ckpt_path)
    # Download the vocabularies.
    src_vocab_file = model_files['src_vocab']
    trg_vocab_file = model_files['trg_vocab']
    src_vocab_path = os.path.join(model_dir, 'src_vocab.txt')
    self._download(src_vocab_file, src_vocab_path)
    trg_vocab_path = os.path.join(model_dir, 'trg_vocab.txt')
    self._download(trg_vocab_file, trg_vocab_path)
    # Download the config.
    config_file = model_files['config.yaml']
    config_path = os.path.join(model_dir, 'config_orig.yaml')
    self._download(config_file, config_path)
    # Adjust config.
    config = load_config(config_path)
    new_config_file = os.path.join(model_dir, 'config.yaml')
    config = self._update_config(config, src_vocab_path, trg_vocab_path,
                                 model_dir, ckpt_path)
    with open(new_config_file, 'w') as cfile:
      yaml.dump(config, cfile)
    # Download BPE codes.
    src_bpe_path = os.path.join(model_dir, 'src.bpe.model')
    trg_bpe_path = os.path.join(model_dir, 'trg.bpe.model')
    self._download(model_files['src_bpe'], src_bpe_path)
    self._download(model_files['trg_bpe'], trg_bpe_path)
    print('Downloaded model for {}-{}.'.format(self._src_language, trg_language))
    return model_dir, config, self._is_lc(src_vocab_path)

  def _update_config(self, config, new_src_vocab_path, new_trg_vocab_path,
                     new_model_dir, new_ckpt_path):
    """Overwrite the settings in the given config."""
    config['data']['src_vocab'] = new_src_vocab_path
    if config['model'].get('tied_embeddings', False):
      config['data']['trg_vocab'] = new_src_vocab_path
    else:
      config['data']['trg_vocab'] = new_trg_vocab_path
    config['training']['model_dir'] = new_model_dir
    config['training']['load_model'] = new_ckpt_path
    return config

  def _is_lc(self, src_vocab_path):
    # Infer whether the model is built on lowercased data.
    lc = True
    with open(src_vocab_path, 'r') as ofile:
      for line in ofile:
        if line != line.lower():
          lc = False
          break
    return lc

  def _download_gdrive_file(self, file_id, destination):
    """Download a file from Google Drive and store in local file."""
    download_link = 'https://drive.google.com/uc?id={}'.format(file_id)
    !gdown -q -O $destination $download_link

  def _download_github_file(self, github_raw_path, destination):
    """Download a file from GitHub."""
    ! wget -q -O $destination $github_raw_path

  def _download(self, url, destination):
    """Download file from Github or Googledrive."""
    try:
      if 'drive.google.com' in url:
        if url.startswith('https://drive.google.com/file'):
          file_id = url.split("/")[-1]
        elif url.startswith('https://drive.google.com/open?'):
          file_id = url.split('id=')[-1]
        self._download_gdrive_file(file_id, destination)
      else:
        self._download_github_file(url, destination)
    except:
      print("Download failed, didn't recognize url {}.".format(url))

available_models_file = 'available_models.tsv'
! wget -q -O $available_models_file https://raw.githubusercontent.com/juliakreutzer/masakhane-covid/master/models/available_models.tsv

model_loader = MasakhaneModelLoader(available_models_file=available_models_file)

import ipywidgets as widgets
print('Please select a target language.')
lang_picker = widgets.Dropdown(options=model_loader.models.keys(), value='yo')
lang_picker

Found 30 Masakhane models.
Please select a target language.


Dropdown(index=25, options=('af', 'ar', 'am', 'ddn', 'efi', 'bin', 'ish', 'fon', 'ha', 'ig', 'iso', 'kam', 'ki…

In [4]:
#@title Model download 
language = lang_picker.value
model_dir, config, lc = model_loader.download_model(language)

Downloaded model for en-twi.


In [5]:
#@title Test loaded model
# Try if this works: this should not lead to an error message. 
! echo "Test.\nAnd again." > test.txt
new_config_path = os.path.join(model_dir, 'config.yaml')
! python -m joeynmt translate $new_config_path < test.txt > test_out.txt

2020-07-30 04:57:00,103 Hello! This is Joey-NMT.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


If the above cell failed, something is wrong with the translation model. Please do not proceed, but email Julia instead with the error message.

## TED Data

Now we're loading the data to translate.

In [6]:
#@title Load the source

class SourceData():
  def __init__(self, data_link, bpe_path, out_file):
    self._src_df = pd.read_csv(data_link, sep='\t', header=None,
                               names=['source'])
    print("Loaded {} lines.".format(len(self._src_df)))
    self._bpe_model = self.load_bpe(bpe_path)
    self._src_df, self._sources = self.preprocess(out_file)
  
  def get_df(self):
    return self._src_df
  
  def get_sources(self):
    return self._sources

  def preprocess(self, out_file):
    """Tokenize, (lowercase,) sub-word split.
    
    Using Polyglot since it was used for JW300.
    Preprocess the source column of a dataframe object and write to file.
  
    Pipeline:
    - tokenize
    - split into sub-words

    Append pre-processed sources to dataframe."""
    tokenized_sentences = []
    bped_sentences = []
    sources = []
    with open(out_file, 'w') as ofile:
      for i, row in self._src_df.iterrows():
        sentence_i = Text(row[0]).sentences[0]
        tokenized_sentence = ""
        bped_sentence = ""
        tokenized = " ".join(sentence_i.words)
        sources.append(str(sentence_i))
        if lc:
          tokenized = tokenized.lower()
        tokenized_sentence = tokenized
        bped = self._bpe_model.process_line(tokenized)
        bped_sentence = bped
        ofile.write("{}\n".format(bped))
        tokenized_sentences.append(tokenized_sentence)
        bped_sentences.append(bped_sentence)
    data = self._src_df.assign(
        tokenized_sentences=tokenized_sentences)
    data = data.assign(
        bped_sentences=bped_sentences)
    return data, sources

  def load_bpe(self, bpe_path):
    with open(bpe_path, 'r') as ofile:
      bpe_model = apply_bpe.BPE(codes=ofile)
    return bpe_model
  
src_input_file = 'src_input.bpe.txt'
src_bpe_path = os.path.join(model_dir, 'src.bpe.model')
ted_link = 'https://raw.githubusercontent.com/juliakreutzer/masakhane-eval/master/data/multitarget-ted-filt.en.tsv'
src_data = SourceData(ted_link, bpe_path=src_bpe_path, out_file=src_input_file)
sources = src_data.get_sources()
ted_df = src_data.get_df()
! sed 's/@@ //g' $src_input_file > src_input.txt

Loaded 1520 lines.


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


In [7]:
#@title Source excerpt
ted_df[:3]

,source,tokenized_sentences,bped_sentences
0,"Reset the three minutes, that's just not fair.","Reset the three minutes , that's just not fair .","R@@ es@@ et the three min@@ ut@@ es , th@@ at@..."
1,I mean I'm nervous enough as it is.,I mean I'm nervous enough as it is .,I mean I@@ '@@ m n@@ er@@ v@@ ous enough as it...
2,But I am not as nervous as I was five weeks ago.,But I am not as nervous as I was five weeks ago .,But I am not as n@@ er@@ v@@ ous as I was five...


In [8]:
#@title Translation with Joey NMT - it can take a few minutes.
trg_output_file = 'targets.txt'
print('Translating...')
! python -m joeynmt translate $model_dir/config.yaml < $src_input_file > $trg_output_file

# Post-processing
def post_process(output_file):
  """Load and detokenize translations.
  
  There is no given Polyglot detokenizer, so we do it by heuristics.
  """
  targets = []
  with open(trg_output_file, 'r') as ofile:
    for line in ofile:
      sent = line.strip()
      sent = sent.replace('<pad>', '')
      sent = re.sub(r'\s+([?.!"-,:’])', r'\1', sent)
      sent = sent.replace('( ', '(').replace(' - ', '-').replace(' / ', '/').replace(' /', '/')
      if lc:
        # Cheap casing restoration... only first character but better than nothing.
        sent = sent[0].upper() + sent[1:]
      targets.append(sent)
  return targets

targets = post_process(trg_output_file)

print('Done!')

Translating...
2020-07-30 05:01:07,534 Hello! This is Joey-NMT.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
Done!


## Post-edit
Let's correct the model's mistakes. Please correct the translations and click on "Next Sentence" until you've done enough. Enough means that you're getting tired, that you spent as much time as you wanted on it, or that the data is exhausted.

If you have previously worked on the translations, and want to start from a line that is not the beginning of the file, please select this line in the following field and we'll start from there. 
We're starting to count at 1.
If you have not worked on this yet, please start by 1.

To save the intermediate progress (e.g. if you want to take a break), execute the cells below the post-edit interface.

In [10]:
#@title Starting position

start_sent = 1 #@param {type:"number"}


If you want preceding sentences to be displayed as well, set ´num_cont_sent´ to the desired number of sentences. Setting it to 0 results in showing no preceding sentences. On the target side, the field shows your prior corrections.

In [25]:
#@title Number of sentences for context

num_cont_sent =  0#@param {type:"number"}
assert num_cont_sent >= 0


In [26]:
#@title Now it's your turn!
class SentenceCounterLabel():
  def __init__(self, num_sentences):
    self._num_sentences = num_sentences
    self._label = widgets.Label("")

  def update(self, current_sentence_index):
    self._label.value = f"Please correct the mistakes in the following translation ({current_sentence_index + 1} out of {self._num_sentences}). Delete the proposed translation if you do not know how to correct or replace it."

  def get_label(self):
    return self._label

class SentenceCounter(object):
  def __init__(self, source_sentences, target_sentences, counter=1):
    self._count = counter-1
    self._init_count = counter-1
    
    assert len(source_sentences) == len(target_sentences)
    self._max_count = len(source_sentences) - 1

    self._source = source_sentences
    self._target = target_sentences
    self._post_edits = [""]*len(target_sentences)

    self._label = SentenceCounterLabel(self._max_count + 1)
    self._update_label()

  def _update_label(self):
    self._label.update(self._count)

  def decrement(self):
    if self._count > 0:
      self._count -= 1

    self._update_label()

  def increment(self):
    if self._count < self._max_count:
      self._count += 1

    self._update_label()

  def get_count(self):
    return self._count
  
  def get_max_count(self):
    return self._max_count

  def get_source(self):
    return self._source[self._count]

  def get_target(self):
    return self._target[self._count]

  def get_target_context(self):
    return '\n'.join(
        self._post_edits[max(self._count-num_cont_sent, 0):self._count])
    
  def get_source_context(self):
    return '\n'.join(
        self._source[max(self._count-num_cont_sent, 0):self._count])

  def update_pe(self, post_edit):
    self._post_edits[self._count] = post_edit

  def get_label(self):
    return self._label.get_label()

  def get_pes(self):
    return self._post_edits

  @property
  def pe_is_complete(self):
    return all([e != '' for e in self._post_edits])

sentence_counter = SentenceCounter(
  sources, targets, counter=start_sent
)

out = widgets.Output()

src_context = widgets.Textarea(
    value=sentence_counter.get_source_context(),
    description="Source context",
    disabled=True,
    layout=widgets.Layout(width="90%", overflow="auto"),
  rows=max(1, num_cont_sent)
)

trg_context = widgets.Textarea(
    value=sentence_counter.get_target_context(),
    description="Target context (corrected)",
    disabled=True,
    layout=widgets.Layout(width="90%", overflow="auto"),
  rows=max(1, num_cont_sent)
)

trg_text = widgets.Textarea(
  value=sentence_counter.get_target(),
  description="Target",
  disabled=False,
  layout=widgets.Layout(width="90%", overflow="auto"),
  rows=5
)

src_text = widgets.Textarea(
  value=sentence_counter.get_source(),
  description="Source",
  disabled=True,
  layout=widgets.Layout(width="90%", overflow="auto"),
  rows=5
)

prev_button = widgets.Button(
  description='Previous Sentence',
  disabled=False,
  button_style='',
  tooltip='Click me',
  icon='check'
)

next_button = widgets.Button(
  description='Next Sentence',
  disabled=False,
  button_style='',
  tooltip='Click me',
  icon='check'
)

def update_text_fields(trg_text_field, src_text_field, trg_context_field,
                       src_context_field, counter, increment=True):
  counter.update_pe(trg_text_field.value)

  if increment:
    counter.increment()

  else:
    counter.decrement()
  
  trg_text_field.value = counter.get_target()
  src_text_field.value = counter.get_source()
  trg_context_field.value = counter.get_target_context()
  src_context_field.value = counter.get_source_context()


with out:
  def on_submit(trg_text_field, src_text_field, trg_context_field,
                src_context_field, counter):
    update_text_fields(trg_text_field, src_text_field, trg_context_field, 
                       src_context_field, counter)

  def on_click(button, trg_text_field, src_text_field, trg_context_field,
                src_context_field, counter, increment=True):
    update_text_fields(trg_text_field, src_text_field, trg_context_field, 
                       src_context_field, counter, increment=increment)

  display(sentence_counter.get_label())

  if num_cont_sent > 0:
    display(src_context)
  display(src_text)
  if num_cont_sent > 0:
    display(trg_context)
  display(trg_text)
  display(prev_button)
  display(next_button)

  prev_button.on_click(functools.partial(
    on_click,
    trg_text_field=trg_text,
    src_text_field=src_text,
    src_context_field=src_context,
    trg_context_field=trg_context,
    counter=sentence_counter,
    increment=False
  ))

  next_button.on_click(functools.partial(
    on_click,
    trg_text_field=trg_text,
    src_text_field=src_text,
    src_context_field=src_context,
    trg_context_field=trg_context,
    counter=sentence_counter
  ))

display(out)

Output()

In [ ]:
print("Finished? You have completed {} sentences. BRAVO!".format(sentence_counter._count-sentence_counter._init_count))
end_sent = sentence_counter._count
post_edits = sentence_counter.get_pes()

Finished? You have completed 5 sentences. BRAVO!


### Post-Edit Analysis

We compute a bunch of automatic metrics to assess how much had to be edited.
- BLEU: on corpus- and on sentence-level, for tokenized inputs and without tokenization.
- ChrF: a character-level metric, on untokenized inputs.
- TER: on sentence-level, white-space tokenized.

In addition, your feedback would be very valuable.


In [ ]:
#@title Where did the model struggle?

# Compute metrics only for selected talks on corpus level, and for everything on sent. level.
sel_hyps, sel_pes = zip(*[(h, p) for h, p in zip(targets, post_edits) if p !=''])
bleu_pe_none_tok = sacrebleu.corpus_bleu(sel_hyps, [sel_pes], tokenize='none').score
bleu_pe_intl_tok = sacrebleu.corpus_bleu(sel_hyps, [sel_pes], tokenize='intl').score
bleu_pe_none_tok_sent = [sacrebleu.sentence_bleu(target, post_edit).score for target, post_edit in zip(targets, post_edits)]
chrf_pe_sent = [sacrebleu.sentence_chrf(target, post_edit).score*100 for target, post_edit in zip(targets, post_edits)]
ter_pe_sent = [pyter.ter(target.split(' '), post_edit.split(' ')) for target, post_edit in zip(targets, post_edits)]

#print('BLEU without tokenization: ', bleu_pe_none_tok)
#print('BLEU with intl tokenization: ', bleu_pe_intl_tok)
#print('Avg sent. BLEU', np.mean(bleu_pe_none_tok_sent))
#print('Avg ChrF', np.mean(chrf_pe_sent))
#print('Avg TER', np.mean(ter_pe_sent))

feedback = widgets.Textarea(
  value='The model had difficulties... ',
  description="Feedback",
  disabled=False,
  layout=widgets.Layout(width="90%", overflow="auto"),
  rows=5
)
feedback

Textarea(value='The model had difficulties... ', description='Feedback', layout=Layout(overflow='auto', width=…

## Store the results

We store sources, post-edits and targets in a per-sentence table, and merge individual sentences back together into the format of the survey.


In [ ]:
#@title Prepare output files
assert len(targets) == len(sources) == len(post_edits)

pe_output_file = 'pe_{}_{}-{}.tsv'.format(language, start_sent, end_sent)
with open(pe_output_file, 'w') as ofile:
  ofile.write('source\ttranslation\tpost-edit\tbleu\tchrf\tter\n')
  for t, s, p, b, c, ter in zip(targets, sources, post_edits, bleu_pe_none_tok_sent, chrf_pe_sent, ter_pe_sent):
    ofile.write('{}\t{}\t{}\t{:.2f}\t{:.2f}\t{:.2f}\n'.format(s, t, p, b, c, ter))

pe_metric_file = 'pe_metrics_{}_{}-{}.tsv'.format(language, start_sent, end_sent)
with open(pe_metric_file, 'w') as ofile:
  ofile.write('Corpus BLEU tok=none:\t{:.2f}\n'.format(bleu_pe_none_tok))
  ofile.write('Corpus BLEU tok=intl:\t{:.2f}\n'.format(bleu_pe_intl_tok))
  ofile.write('Sent. BLEU tok=none:\t{:.2f}\n'.format(np.mean(bleu_pe_none_tok_sent)))
  ofile.write('Avg TER tok=white:\t{:.2f}\n'.format(np.mean(ter_pe_sent)))
  ofile.write('Avg ChrF:\t{:.2f}\n'.format(np.mean(chrf_pe_sent)))

feedback_file = 'feedback_{}_{}-{}.txt'.format(language, start_sent, end_sent)
with open(feedback_file, 'w') as ofile:
  ofile.write(feedback.value)

ted_df = ted_df.assign(translations=targets)
ted_df = ted_df.assign(post_edits=post_edits)

def store_translations(data, output_file):
  """Store the translations (post-edited) in a file like the survey inputs."""
  with open(output_file, 'w') as ofile:
    for translation in data['translations']:
      ofile.write('{}\n'.format(translation))

output_file = 'ted_{}_{}-{}.tsv'.format(language, start_sent, end_sent)
store_translations(ted_df, output_file)

In [ ]:
#@title Source and target excerpt
ted_df[start_sent-1:start_sent+4]

,source,tokenized_sentences,bped_sentences,translations,post_edits
0,"Reset the three minutes, that's just not fair.","Reset the three minutes , that's just not fair .","Res@@ et the three minutes , th@@ at@@ '@@ s j...","Nga murahu ha mimunithi miraru, muambuli wo vh...","Nga murahu ha mimunithi miraru, muambuli wo vh..."
1,I mean I'm nervous enough as it is.,I mean I'm nervous enough as it is .,I mean I@@ '@@ m nervous enough as it is .,Ndo vha ndi tshi khou amba nga ha u tshuwa ho ...,Ndo vha ndi tshi khou amba nga ha u tshuwa ho ...
2,But I am not as nervous as I was five weeks ago.,But I am not as nervous as I was five weeks ago .,But I am not as nervous as I was five weeks ago .,Fhedzi a tho ngo tshuwa musi ndi na miṅwaha mi...,Fhedzi a tho ngo tshuwa musi ndi na miṅwaha mi...
3,Five weeks ago I had total hip replacement sur...,Five weeks ago I had total hip replacement sur...,Five weeks ago I had total hip replacement sur...,Ho no fhela vhege ṱhanu dzo fhiraho ndo vha nd...,Ho no fhela vhege ṱhanu dzo fhiraho ndo vha nd...
4,"Electric saw, power drill, totally disgusting ...","Electric saw , power drill , totally disgustin...","El@@ ec@@ tri@@ c saw , power dr@@ ill , total...","Elecc o vhona maanḓa mahulwane, a tshi khou l...","Elecc o vhona maanḓa mahulwane, a tshi khou l..."


# Done!

To finish, complete the following steps
1. Download the following files from this colab:
  - ted_{language}_{start-end}.tsv
  - pe_metrics_{language}_{start-end}.tsv
  - pe_{language}_{start-end}.tsv
  - feedback_{language}_{start-end}.txt
 
  For the download, either execute the next cell and allow the download, or click on 'Files' on the left bar, right-click on files and download.
2. Send the data to kreutzer@cl.uni-heidelberg.de.

In [ ]:
#@title File download
from google.colab import files
files.download('ted_{}_{}-{}.tsv'.format(language, start_sent, end_sent))
files.download('pe_metrics_{}_{}-{}.tsv'.format(language, start_sent, end_sent))
files.download('pe_{}_{}-{}.tsv'.format(language, start_sent, end_sent))
files.download('feedback_{}_{}-{}.txt'.format(language, start_sent, end_sent))

In [ ]:
print('If you continue with more sentences, please start at sentence {}.'.format(end_sent+1))

If you continue with more sentences, please start at sentence 6.
